In [1]:
# run command 'conda install scikit-image' on project's v-env
# run command 'conda install pandas' on project's v-env

In [2]:
DEBUG_TRAIN = False
DEBUG_DATA = False

In [3]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [4]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [5]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

In [6]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [7]:
from torchvision import transforms
csv_file = "./image-data-256/labels-map.csv"
root_dir = "./image-data-256/hangul-images"
batch_size=64
learning_rate=0.0002
num_epoch=1000

korean_dataset = KoreanHandwritingDataset(csv_file,root_dir, transform=transforms.Compose([ToTensor()]))
#dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

In [8]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        out = self.sigmoid(out)
        if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

In [9]:
import torch.optim as optim
model=CNN().double()
#loss_function=nn.BCEWithLogitsLoss()
loss_function=nn.MultiLabelSoftMarginLoss()
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

> ## Training

In [10]:
import matplotlib.pyplot as plt

loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().numpy())

epoch:  1 sequence:  0 tensor(0.7284, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  2 sequence:  0 tensor(0.6996, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  3 sequence:  0 tensor(0.6136, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  4 sequence:  0 tensor(0.5996, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  5 sequence:  0 tensor(0.5988, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  6 sequence:  0 tensor(0.5988, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  7 sequence:  0 tensor(0.5982, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  8 sequence:  0 tensor(0.5971, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  9 sequence:  0 tensor(0.5974, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  10 sequence:  0 tensor(0.5973, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  11 sequence:  0 tensor(0.5973, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  12 sequence:  0 tensor(0.5973, dtype=torch.float64, grad_fn=<MeanB

epoch:  98 sequence:  0 tensor(0.5795, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  99 sequence:  0 tensor(0.5794, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  100 sequence:  0 tensor(0.5792, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  101 sequence:  0 tensor(0.5791, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  102 sequence:  0 tensor(0.5789, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  103 sequence:  0 tensor(0.5788, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  104 sequence:  0 tensor(0.5786, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  105 sequence:  0 tensor(0.5785, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  106 sequence:  0 tensor(0.5783, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  107 sequence:  0 tensor(0.5782, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  108 sequence:  0 tensor(0.5781, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  109 sequence:  0 tensor(0.5779, dtype=torch.floa

epoch:  194 sequence:  0 tensor(0.5631, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  195 sequence:  0 tensor(0.5624, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  196 sequence:  0 tensor(0.5625, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  197 sequence:  0 tensor(0.5622, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  198 sequence:  0 tensor(0.5622, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  199 sequence:  0 tensor(0.5619, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  200 sequence:  0 tensor(0.5617, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  201 sequence:  0 tensor(0.5615, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  202 sequence:  0 tensor(0.5614, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  203 sequence:  0 tensor(0.5612, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  204 sequence:  0 tensor(0.5612, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  205 sequence:  0 tensor(0.5610, dtype=torch.fl

epoch:  290 sequence:  0 tensor(0.5570, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  291 sequence:  0 tensor(0.5577, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  292 sequence:  0 tensor(0.5577, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  293 sequence:  0 tensor(0.5571, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  294 sequence:  0 tensor(0.5568, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  295 sequence:  0 tensor(0.5566, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  296 sequence:  0 tensor(0.5562, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  297 sequence:  0 tensor(0.5559, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  298 sequence:  0 tensor(0.5557, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  299 sequence:  0 tensor(0.5556, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  300 sequence:  0 tensor(0.5555, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  301 sequence:  0 tensor(0.5555, dtype=torch.fl

epoch:  386 sequence:  0 tensor(0.5519, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  387 sequence:  0 tensor(0.5519, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  388 sequence:  0 tensor(0.5517, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  389 sequence:  0 tensor(0.5515, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  390 sequence:  0 tensor(0.5515, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  391 sequence:  0 tensor(0.5516, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  392 sequence:  0 tensor(0.5516, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  393 sequence:  0 tensor(0.5518, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  394 sequence:  0 tensor(0.5521, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  395 sequence:  0 tensor(0.5524, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  396 sequence:  0 tensor(0.5523, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  397 sequence:  0 tensor(0.5519, dtype=torch.fl

...

epoch:  578 sequence:  0 tensor(0.5435, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  579 sequence:  0 tensor(0.5436, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  580 sequence:  0 tensor(0.5436, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  581 sequence:  0 tensor(0.5435, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  582 sequence:  0 tensor(0.5436, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  583 sequence:  0 tensor(0.5436, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  584 sequence:  0 tensor(0.5441, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  585 sequence:  0 tensor(0.5443, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  586 sequence:  0 tensor(0.5442, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  587 sequence:  0 tensor(0.5443, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  588 sequence:  0 tensor(0.5438, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  589 sequence:  0 tensor(0.5436, dtype=torch.fl

epoch:  674 sequence:  0 tensor(0.5401, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  675 sequence:  0 tensor(0.5401, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  676 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  677 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  678 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  679 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  680 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  681 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  682 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  683 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  684 sequence:  0 tensor(0.5400, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  685 sequence:  0 tensor(0.5400, dtype=torch.fl

epoch:  770 sequence:  0 tensor(0.5352, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  771 sequence:  0 tensor(0.5353, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  772 sequence:  0 tensor(0.5357, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  773 sequence:  0 tensor(0.5351, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  774 sequence:  0 tensor(0.5346, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  775 sequence:  0 tensor(0.5348, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  776 sequence:  0 tensor(0.5352, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  777 sequence:  0 tensor(0.5392, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  778 sequence:  0 tensor(0.5392, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  779 sequence:  0 tensor(0.5367, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  780 sequence:  0 tensor(0.5347, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  781 sequence:  0 tensor(0.5354, dtype=torch.fl

epoch:  866 sequence:  0 tensor(0.5331, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  867 sequence:  0 tensor(0.5331, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  868 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  869 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  870 sequence:  0 tensor(0.5334, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  871 sequence:  0 tensor(0.5331, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  872 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  873 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  874 sequence:  0 tensor(0.5331, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  875 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  876 sequence:  0 tensor(0.5332, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  877 sequence:  0 tensor(0.5332, dtype=torch.fl

epoch:  962 sequence:  0 tensor(0.5316, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  963 sequence:  0 tensor(0.5315, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  964 sequence:  0 tensor(0.5316, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  965 sequence:  0 tensor(0.5314, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  966 sequence:  0 tensor(0.5315, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  967 sequence:  0 tensor(0.5316, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  968 sequence:  0 tensor(0.5314, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  969 sequence:  0 tensor(0.5315, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  970 sequence:  0 tensor(0.5315, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  971 sequence:  0 tensor(0.5314, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  972 sequence:  0 tensor(0.5314, dtype=torch.float64, grad_fn=<MeanBackward0>)
epoch:  973 sequence:  0 tensor(0.5314, dtype=torch.fl

In [15]:
PATH = "./saved_model/MultiLabelSoftMarginLoss-1000epoch.torchModel"
torch.save(model.state_dict(), PATH)

In [16]:
loaded_model = CNN().double()
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.eval()

CNN(
  (avgpool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1152, out_features=378, bias=True)
  (relu3): ReLU()
  (linear2): Linear(in_features=378, out_features=128, bias=True)
  (relu4): ReLU()
  (linear3): Linear(in_features=128, out_features=24, bias=True)
  (sigmoid): Sigmoid()
)

> ## Testing

In [11]:
with torch.no_grad():
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        
        output = model.forward(x)
        for i in range(len(output)):
            print("출력 값: ", decode_from_bin(output[i].tolist()),
                  ", 실제 값: ", decode_from_bin(y[i].tolist()))
            #print("실제 값: ", output[i].tolist(),
            #      ", 출력 값: ", y[i].tolist())

출력 값:  끤 , 실제 값:  다
출력 값:  끤 , 실제 값:  다
출력 값:  끤 , 실제 값:  다
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  쒀 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒀 , 실제 값:  자
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  저 , 실제 값:  적
출력 값:  저 , 실제 값:  적
출력 값:  저 , 실제 값:  적
출력 값:  ꠀ , 실제 값:  적
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아


출력 값:  퀴 , 실제 값:  피
출력 값:  퐼 , 실제 값:  피
출력 값:  퀼 , 실제 값:  피
출력 값:  낰 , 실제 값:  편
출력 값:  낰 , 실제 값:  편
출력 값:  낰 , 실제 값:  편
출력 값:  낰 , 실제 값:  편
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  룤 , 실제 값:  매
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  근 , 실제 값:  근
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  뀰 , 실제 값:  터
출력 값:  쓄 , 실제 값:  업
출력 값:  쓄 , 실제 값:  업
출력 값:  쓄 , 실제 값:  업
출력 값:  쓄 , 실제 값:  업
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  렀 , 실제 값:  버
출력 값:  쀌 , 실제 값:  석
출력 값:  쀌 , 실제 값:  석
출력 값:  쀌 , 실제 값:  석
출력 값:  쀌 , 실제 값:  석
출력 값:  냄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  듄 , 실제 값:  들
출력 값:  저 , 실제 값:  절
출력 값:  저 , 실제 값:  절
출력 값:  저 , 실제 값:  절
출력 값:  저 , 실제 값:  절
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  갰 , 실제 값:  결
출력 값:  쑼 , 실제 값:  약
출력 값:  쑼 , 실제 값:  약
출력 값:  쑼 , 실제 값:  약
출력 값:  쑼 , 실제 값:  약
출력 값:  죀 , 실제 값:  직
출력 값:  죀 , 실제 값:  직
출력 값:  죀 , 실제 값:  직


> ## Testing Loaded Model

In [17]:
with torch.no_grad():
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        
        output = loaded_model.forward(x)
        for i in range(len(output)):
            print("출력 값: ", decode_from_bin(output[i].tolist()),
                  ", 실제 값: ", decode_from_bin(y[i].tolist()))
            
            #print("실제 값: ", output[i].tolist(),
            #      ", 출력 값: ", y[i].tolist())

출력 값:  끤 , 실제 값:  다
출력 값:  끤 , 실제 값:  다
출력 값:  끤 , 실제 값:  다
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  푐 , 실제 값:  하
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  죀 , 실제 값:  지
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  왴 , 실제 값:  이
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  갰 , 실제 값:  기
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  렬 , 실제 값:  리
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  가 , 실제 값:  가
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  삌 , 실제 값:  사
출력 값:  쒀 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒐 , 실제 값:  자
출력 값:  쒀 , 실제 값:  자
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  뀀 , 실제 값:  대
출력 값:  저 , 실제 값:  적
출력 값:  저 , 실제 값:  적
출력 값:  저 , 실제 값:  적
출력 값:  ꠀ , 실제 값:  적
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쐴 , 실제 값:  어
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아
출력 값:  쑄 , 실제 값:  아


출력 값:  천 , 실제 값:  출
출력 값:  품 , 실제 값:  행
출력 값:  품 , 실제 값:  행
출력 값:  품 , 실제 값:  행
출력 값:  품 , 실제 값:  행
출력 값:  삐 , 실제 값:  산
출력 값:  삐 , 실제 값:  산
출력 값:  삐 , 실제 값:  산
출력 값:  쀰 , 실제 값:  산
출력 값:  뒘 , 실제 값:  래
출력 값:  나 , 실제 값:  래
출력 값:  되 , 실제 값:  래
출력 값:  뒘 , 실제 값:  래
출력 값:  죀 , 실제 값:  진
출력 값:  죀 , 실제 값:  진
출력 값:  죀 , 실제 값:  진
출력 값:  죀 , 실제 값:  진
출력 값:  쒐 , 실제 값:  양
출력 값:  쒐 , 실제 값:  양
출력 값:  쒐 , 실제 값:  양
출력 값:  쒐 , 실제 값:  양
출력 값:  회 , 실제 값:  회
출력 값:  회 , 실제 값:  회
출력 값:  회 , 실제 값:  회
출력 값:  회 , 실제 값:  회
출력 값:  뢀 , 실제 값:  명
출력 값:  뢀 , 실제 값:  명
출력 값:  뢀 , 실제 값:  명
출력 값:  뢀 , 실제 값:  명
출력 값:  쒌 , 실제 값:  재
출력 값:  쒌 , 실제 값:  재
출력 값:  쒌 , 실제 값:  재
출력 값:  쒌 , 실제 값:  재
출력 값:  냘 , 실제 값:  당
출력 값:  냐 , 실제 값:  당
출력 값:  냘 , 실제 값:  당
출력 값:  냘 , 실제 값:  당
출력 값:  려 , 실제 값:  려
출력 값:  려 , 실제 값:  려
출력 값:  려 , 실제 값:  려
출력 값:  려 , 실제 값:  려
출력 값:  찀 , 실제 값:  초
출력 값:  찀 , 실제 값:  초
출력 값:  찀 , 실제 값:  초
출력 값:  찀 , 실제 값:  초
출력 값:  첔 , 실제 값:  체
출력 값:  첔 , 실제 값:  체
출력 값:  첔 , 실제 값:  체
출력 값:  첔 , 실제 값:  체
출력 값:  룐 , 실제 값:  말


출력 값:  쒀 , 실제 값:  잡
출력 값:  쒀 , 실제 값:  잡
출력 값:  쒀 , 실제 값:  잡
출력 값:  쒀 , 실제 값:  잡
출력 값:  벀 , 실제 값:  벌
출력 값:  벀 , 실제 값:  벌
출력 값:  벀 , 실제 값:  벌
출력 값:  밀 , 실제 값:  벌
출력 값:  욀 , 실제 값:  움
출력 값:  욀 , 실제 값:  움
출력 값:  욀 , 실제 값:  움
출력 값:  움 , 실제 값:  움
출력 값:  처 , 실제 값:  처
출력 값:  처 , 실제 값:  처
출력 값:  처 , 실제 값:  처
출력 값:  첐 , 실제 값:  처
출력 값:  푈 , 실제 값:  합
출력 값:  푈 , 실제 값:  합
출력 값:  푈 , 실제 값:  합
출력 값:  푈 , 실제 값:  합
출력 값:  쀨 , 실제 값:  씨
출력 값:  쀨 , 실제 값:  씨
출력 값:  쀨 , 실제 값:  씨
출력 값:  쀨 , 실제 값:  씨
출력 값:  킀 , 실제 값:  토
출력 값:  킀 , 실제 값:  토
출력 값:  킀 , 실제 값:  토
출력 값:  킀 , 실제 값:  토
출력 값:  풄 , 실제 값:  향
출력 값:  풄 , 실제 값:  향
출력 값:  풄 , 실제 값:  향
출력 값:  풄 , 실제 값:  향
출력 값:  삌 , 실제 값:  삼
출력 값:  삌 , 실제 값:  삼
출력 값:  삌 , 실제 값:  삼
출력 값:  삌 , 실제 값:  삼
출력 값:  변 , 실제 값:  변
출력 값:  변 , 실제 값:  변
출력 값:  변 , 실제 값:  변
출력 값:  변 , 실제 값:  변
출력 값:  낀 , 실제 값:  능
출력 값:  낀 , 실제 값:  능
출력 값:  낀 , 실제 값:  능
출력 값:  낀 , 실제 값:  능
출력 값:  냄 , 실제 값:  답
출력 값:  냄 , 실제 값:  답
출력 값:  냄 , 실제 값:  답
출력 값:  냄 , 실제 값:  답
출력 값:  옠 , 실제 값:  육
출력 값:  옠 , 실제 값:  육
